In [ ]:
import os, re, unicodedata
from datetime import datetime, timedelta
from pyspark.sql import SparkSession, functions as F
from notebookutils import mssparkutils

In [ ]:
# ---------- Spark ----------
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Seleccionar el Lakehouse destino (bronze)
spark.sql("USE lh_bronze")

In [ ]:
# 📂 Carpeta base donde se guardan los JSON diarios del proceso de ingestión
daily_json_base = "Files/bronze/ESIOS/data/mercados/componentes-precio/day/"+str(datetime.now().year)+"/daily"
daily_json_base_physical = "/lakehouse/default/Files/bronze/ESIOS/data/mercados/componentes-precio/day/"+str(datetime.now().year)+"/daily"


In [ ]:
# --------------------------
# Función utilitaria mejorada
# --------------------------
def slugify(name: str) -> str:
    # Normalizar y quitar tildes
    name = unicodedata.normalize("NFKD", name)
    name = name.encode("ascii", "ignore").decode("utf-8")  # quita acentos
    # Sustituir caracteres no válidos por "_"
    return re.sub(r'[^a-z0-9_]', '_', name.lower())

In [ ]:
# --------------------------
# Procesamiento de JSON diarios
# --------------------------
total_rows = 0
processed_files = 0

# Obtener el año actual para procesar solo los datos del año en curso
current_year = datetime.now().year
print(f"📅 Año actual: {current_year}")
print(f"📂 Directorio de procesamiento: {daily_json_base}")

# Listar archivos JSON directamente en el directorio daily del año actual
try:
    json_files = mssparkutils.fs.ls(daily_json_base)
    print(f"📄 Archivos JSON encontrados: {len(json_files)}")
    for f in json_files:
        print(f"   - {f.path}")
except Exception as e:
    print(f"❌ Error al listar archivos en {daily_json_base}: {e}")
    json_files = []

for f in json_files:
    if not f.path.endswith(".json"):
        continue

    print(f"🔎 Procesando {f.path}")

    try:
        # Leer JSON
        df_raw = spark.read.option("multiline", True).json(f.path)
        
        # Verificar si el JSON tiene la estructura esperada
        if "indicator" not in df_raw.columns:
            print(f"⚠️ Estructura JSON inesperada en {f.path}, saltando")
            continue

        # Extraer indicador, values
        indicator_id = df_raw.select("indicator.id").first()[0]
        indicator_type = df_raw.select("indicator.short_name").first()[0] \
                      if "short_name" in df_raw.select("indicator.*").columns else indicator_id

        values = df_raw.selectExpr("explode(indicator.values) as val") \
                   .select("val.*")

        if values.rdd.isEmpty():
            print(f"⚠️ JSON vacío (sin values) en {f.path}, saltando")
            continue

        # Enriquecer con metadatos
        df = (values
              .withColumn("indicator_id", F.lit(indicator_id))
              .withColumn("indicator_type", F.lit(indicator_type))
              .withColumn("year", F.lit(current_year))
              .withColumn("time_trunc", F.lit("day"))  # Siempre es daily
             )

        # Nombre tabla → según tipo de indicador
        indicator_slug = slugify(indicator_type)
        table_name = f"brz_esios_mercados_{indicator_slug}_day"

        # Verificar si la tabla existe antes de intentar append
        table_exists = spark._jsparkSession.catalog().tableExists("lh_bronze", table_name)
        
        if not table_exists:
            print(f"⚠️ La tabla {table_name} no existe. Creándola con overwrite.")
            df.write.format("delta").mode("overwrite").saveAsTable(table_name)
        else:
            # Guardar en Delta gestionado (append a tabla existente)
            df.write.format("delta").mode("append").saveAsTable(table_name)

        count = df.count()
        total_rows += count
        processed_files += 1
        print(f"✅ Guardado en tabla {table_name} ({count} registros)")

    except Exception as e:
        print(f"❌ Error procesando archivo {f.path}: {e}")
        continue

print(f"📊 Proceso terminado - {datetime.utcnow().isoformat()}Z")
print(f"Total archivos procesados: {processed_files}")
print(f"Total registros procesados: {total_rows}")

if processed_files == 0:
    print("ℹ️  No se encontraron archivos JSON para procesar.")
    print("ℹ️  Verifica que:")
    print("   1. El notebook de ingestión diaria se ejecutó correctamente")
    print("   2. Los archivos JSON están en la ruta: {daily_json_base}")
    print("   3. Los archivos tienen nombres como: brz-mercados-pvpc_t_2_0td-daily.json")
